<a href="https://colab.research.google.com/github/Mageed-Ghaleb/OptimizationSystems-Course/blob/main/solving_LPs_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install pyomo highspy

In [3]:
# from pyomo.environ import (
#     ConcreteModel, Var, Objective, Constraint,
#     NonNegativeReals, Binary, minimize, value, Set
# )
# from pyomo.opt import SolverFactory

In [2]:
import pyomo.environ as pyo

m = pyo.ConcreteModel()
m.x1 = pyo.Var(domain=pyo.NonNegativeReals)
m.x2 = pyo.Var(domain=pyo.NonNegativeReals)

# Maximize 3x1 + 2x2
m.obj = pyo.Objective(expr=3*m.x1 + 2*m.x2, sense=pyo.maximize)

# Constraints
m.c1 = pyo.Constraint(expr=2*m.x1 + 1*m.x2 <= 100)
m.c2 = pyo.Constraint(expr=1*m.x1 + 1*m.x2 <= 80)
m.c3 = pyo.Constraint(expr=m.x1 <= 40)

# Tell Pyomo to import extra solver info (duals, reduced costs)
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)
m.rc   = pyo.Suffix(direction=pyo.Suffix.IMPORT)    # reduced costs (vars)
m.slack= pyo.Suffix(direction=pyo.Suffix.IMPORT)    # slacks (constraints)

# Solve
solver = pyo.SolverFactory("highs")
res = solver.solve(m, tee=True)

# --- Primal solution
print("Status:", res.solver.status)
print("Termination:", res.solver.termination_condition)
print("Objective:", pyo.value(m.obj))
print("x1:", pyo.value(m.x1), "x2:", pyo.value(m.x2))


# --- Slacks (b - Ax for <= constraints)
for name, con in [("finish", m.c1), ("carp", m.c2), ("capS", m.c3)]:
    print(f"Slack[{name}] =", m.slack.get(con, None))

# --- Duals (shadow prices)
for name, con in [("finish", m.c1), ("carp", m.c2), ("capS", m.c3)]:
    print(f"Dual[{name}] =", m.dual.get(con, None))

# --- Reduced costs
for name, var in [("S", m.x1), ("T", m.x2)]:
    print(f"RC[{name}] =", m.rc.get(var, None))

Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 3 rows; 2 cols; 5 nonzeros
Coefficient ranges:
  Matrix  [1e+00, 2e+00]
  Cost    [2e+00, 3e+00]
  Bound   [0e+00, 0e+00]
  RHS     [4e+01, 1e+02]
Presolving model
2 rows, 2 cols, 4 nonzeros  0s
2 rows, 2 cols, 4 nonzeros  0s
Presolve reductions: rows 2(-1); columns 2(-0); nonzeros 4(-1) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.9999937513e+00 Ph1: 2(2); Du: 1(1.99999) 0s
          2    -1.8000000000e+02 Pr: 0(0) 0s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 2
Objective value     :  1.8000000000e+02
P-D objective error :  0.0000000000e+00
HiGHS run time      :          0.01
Status: ok
Termination: optimal
Objective: 180.0
x1: 20.0 x2: 60.0
Slack[finish] = None
Slack[carp] = None
Slack[capS] = None
Dual[fi

In [3]:
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

res = solver.solve(m, tee=False)

for cname in ["c1","c2","c3"]:
    c = getattr(m, cname)
    print(cname, "dual =", m.dual.get(c, None))

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


c1 dual = 1.0
c2 dual = 1.0
c3 dual = 0.0


In [4]:
import numpy as np
from scipy.optimize import linprog

# Max 3S+2T  <=>  Min -3S -2T
c = np.array([-3, -2])

A_ub = np.array([
    [2, 1],   # 2S + T <= 100
    [1, 1],   # S + T <= 80
    [1, 0],   # S <= 40
], dtype=float)

b_ub = np.array([100, 80, 40], dtype=float)

bounds = [(0, None), (0, None)]

res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")

print("S,T =", res.x)
print("Max objective =", -res.fun)
print("Slacks (b - Ax) =", res.slack)

# Marginals (dual values / shadow prices) for inequality constraints:
print("Inequality marginals =", res.ineqlin.marginals)

# Bound marginals (reduced-cost-like info):
print("Lower bound marginals =", res.lower.marginals)
print("Upper bound marginals =", res.upper.marginals)

S,T = [20. 60.]
Max objective = 180.0
Slacks (b - Ax) = [ 0.  0. 20.]
Inequality marginals = [-1. -1. -0.]
Lower bound marginals = [0. 0.]
Upper bound marginals = [0. 0.]


In [5]:
def solve_toys(b1=100, b2=80, b3=40):
    c = np.array([-3, -2])
    A_ub = np.array([[2,1],[1,1],[1,0]], dtype=float)
    b_ub = np.array([b1,b2,b3], dtype=float)
    bounds = [(0,None),(0,None)]
    r = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method="highs")
    return r

base = solve_toys()
base_x = base.x
base_obj = -base.fun
base_marg = base.ineqlin.marginals  # constraint marginals
print("Base x:", base_x, "Base obj:", base_obj, "Base marg:", base_marg)

def sweep_rhs(which, values):
    out = []
    for v in values:
        b = [100,80,40]
        b[which] = float(v)
        r = solve_toys(*b)
        out.append((v, r.status, -r.fun, r.x, r.slack, r.ineqlin.marginals))
    return out

# Sweep finishing RHS (constraint 0) around 100
vals = np.linspace(80, 120, 21)
sweep0 = sweep_rhs(0, vals)

# Print a compact "report" line for each RHS value
for v, status, obj, x, slack, marg in sweep0:
    if status == 0:
        print(f"b1={v:6.1f}  obj={obj:7.2f}  x={x}  slack={slack}  marg={marg}")
    else:
        print(f"b1={v:6.1f}  status={status} (not optimal)")

Base x: [20. 60.] Base obj: 180.0 Base marg: [-1. -1. -0.]
b1=  80.0  obj= 160.00  x=[-0. 80.]  slack=[ 0.  0. 40.]  marg=[-1. -1. -0.]
b1=  82.0  obj= 162.00  x=[ 2. 78.]  slack=[ 0.  0. 38.]  marg=[-1. -1. -0.]
b1=  84.0  obj= 164.00  x=[ 4. 76.]  slack=[ 0.  0. 36.]  marg=[-1. -1. -0.]
b1=  86.0  obj= 166.00  x=[ 6. 74.]  slack=[ 0.  0. 34.]  marg=[-1. -1. -0.]
b1=  88.0  obj= 168.00  x=[ 8. 72.]  slack=[ 0.  0. 32.]  marg=[-1. -1. -0.]
b1=  90.0  obj= 170.00  x=[10. 70.]  slack=[ 0.  0. 30.]  marg=[-1. -1. -0.]
b1=  92.0  obj= 172.00  x=[12. 68.]  slack=[ 0.  0. 28.]  marg=[-1. -1. -0.]
b1=  94.0  obj= 174.00  x=[14. 66.]  slack=[ 0.  0. 26.]  marg=[-1. -1. -0.]
b1=  96.0  obj= 176.00  x=[16. 64.]  slack=[ 0.  0. 24.]  marg=[-1. -1. -0.]
b1=  98.0  obj= 178.00  x=[18. 62.]  slack=[ 0.  0. 22.]  marg=[-1. -1. -0.]
b1= 100.0  obj= 180.00  x=[20. 60.]  slack=[ 0.  0. 20.]  marg=[-1. -1. -0.]
b1= 102.0  obj= 182.00  x=[22. 58.]  slack=[ 0.  0. 18.]  marg=[-1. -1. -0.]
b1= 104.0  obj= 1